### Lec 12: RNN 

- RNN은 sequence data를 처리하는 모델이다. sequence는 순서대로 처리해야 하는 것을 뜻하고, 이런 데이터에는 음성인식, 자연어처리 등이 포함된다. 
- 자연어의 경우 단어 하나만 안다고 해서 처리될 수 없고, 앞뒤 문맥을 함께 이해해야 해석이 가능하다

- 연속적인 값들을 다루기 때문에 state라는 개념이 들어간다. 
- 이전의(old state) 값이 현재(new state) 값에 영향을 미친다. 
- 하나의 함수와 동일한 parameters가 모든 states에 동일하게 적용된다. 
- t는 sequence data에 대한 특정 시점의 데이터를 가리키고, 여기서는 t에 대해 두 가지를 계산한다. 
- 첫 번째 줄의 공식은 W의 이전 상태와 입력을 갖고 fw에 해당하는 tanh 함수를 호출하는 것이다. 
- ht는 현재 상태를 의미하고 h의 t-1번째는 이전(old) 상태와 입력 값(x)을 사용해서 계산한다. 
- yt는 예측 값을 의미하고, W와 현재 상태(ht)를 곱해서 계산한다.

<left><img src= 'img/rnn_1.PNG' width="70%"></left>

##### <font color = 'yellow'> Ex. 4가지 종류의 글자 h, e, l, o가 있고, 연속된 sequence인 "hello"에 대해 이후에 나올 값을 예측하려함 </font>

- 4가지 종류의 글자가 있기 때문에 크기가 4인 벡터(리스트)로 처리
- h의 값과 x의 값을 W와 계산한 다음 tanh 함수에 전달하면 hidden layer에서 보여주는 값이 차례대로 생성됨 
- 매번 계산이 끝날 때마다 새로운 상태를 가리키는 hidden layer의 값이 변화함  
- tanh 함수는 sigmoid 함수 중의 하나로 처음 나왔던 sigmoid를 개량한 버전 
- 기존의 sigmoid가 0에서 1 사이의 값을 반환하는데, tanh 함수는 -1에서 1 사이의 값을 반환하도록 개량. 현재 상태를 가리키는 ht는 tanh 함수의 반환값이므로 -1과 1 사이의 값이 된며, 따라서 hidden layer에 있는 값들도 해당 범위에 존재하게 됨 
- 최종적으로는 모든 단계에서 값을 예측하고 실제 값과 맞는지 비교

<left><img src= 'img/rnn_2.PNG' width="70%"></left>

##### <font color = 'yellow'> RNN의 주요 활용분야 </font>
- 그림1. 바닐라 RNN을 가리킨다. 가장 단순한 형태로 1대1(one-to-one) 기반의 모델

- 그림2. 1대다(one-to-many) 기반의 모델로 이미지에 대해 설명을 붙일 때 사용한다.

  *ex. 한 장의 그림에 대해 "소년이 사과를 고르고 있다"는 자막을 붙일때*

- 그림3. 다대1(many-to-one) 형태의 모델로 여러 개의 입력에 대해 하나의 결과를 만들어 준다. 

  *ex. 우리가 하는 말을 통해 우리의 심리 상태를 "안정", "불안", "공포" 등의 한 단어로 결과를 예측*

- 그림4. 다대다(many-to-many) 형태의 모델로 기계 번역에서 사용

- 그림5. 다대다(many-to-many) 모델의 또 다른 형태.
  
  *ex. 동영상 같은 경우는 여러 개의 이미지 프레임에 대해 여러 개의 설명이나 번역 형태로 결과를 반환* 

<left><img src= 'img/rnn_3.PNG' width="70%"></left>

### Lab 12-1: RNN Basic

In [1]:
import tensorflow as tf
tf.__version__

'1.15.0'

- RNN 모형의 구성은 cell과 output으로 이루어 진다. 
- cell은 적용할 로직(BasicRNN 또는 Basic LSTM 등)과 hidden layer 갯수를 결정 
- output은 데이터와 cell 값을 입력받아, 출력값과 states값을 반환 

In [2]:
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

import warnings 
warnings.filterwarnings(action='ignore')

<left><img src= 'img/one_cell.PNG' width="30%"></left>

In [3]:
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2            ### 레이어가 2개이므로, 2개의 결과값을 반환할 것이다. 
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[[1,0,0,0]]], dtype=np.float32) 
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
array([[[-0.22140579,  0.32247472]]], dtype=float32)


<left><img src= 'img/swquence5.PNG' width="70%"></left>

In [5]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.23053104,  0.6791696 ],
        [-0.16352612,  0.3833307 ],
        [ 0.7607948 , -0.65091115],
        [-0.22333062, -0.12599911],
        [ 0.34676367, -0.0395067 ]]], dtype=float32)


- Batch를 사용하여 많은 데이터를 한번에 입력 

<left><img src= 'img/batch.PNG' width="70%"></left>

In [6]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
array([[[-0.0494266 ,  0.13403285],
        [-0.03541628, -0.06928813],
        [-0.16191316, -0.07597242],
        [-0.2210239 , -0.1117771 ],
        [-0.21533628, -0.07105639]],

       [[-0.01348959, -0.1945803 ],
        [-0.12799458, -0.05386541],
        [-0.210

### Lab 12-2: RNN ~ Hi Hello training 
- 입력된 문자의 다음에 어떤 문자가 나올지를 예측하는 문제 
- hidden_size = 5, input_dim = 5 ~ hihello에서 유니크한 문자 갯수는 5개, 즉 예측해야 하는 값 또한 5개의 문자 
- input_dim = 5 ~ hihello에서 유니크한 문자 갯수는 5개, 이것을 one-hot 코딩으로 변환하면 5개의 컬럼을 가진다.  
- sequence_length = 6 ~ 6개의 문자를 예측/출력해야 한다. 
- batch_size = 1 ~ 한개의 문자열을 다룬다. 

<left><img src= 'img/hihello.PNG' width="70%"></left>

In [7]:
#### 문자열 hihello를 데이터로 변환한다. 
idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello : 최종적으로 얻어야 하는 값 


#### hyper parameters 
num_classes = 5
input_dim = 5          # one-hot size
hidden_size = 5        # output from the LSTM. 5 to directly predict one-hot
batch_size = 1         # one sentence
sequence_length = 6    # |ihello| == 6
learning_rate = 0.1

#### 변수할당, cell과 output 정의, 초기값은 0 
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)


# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b
outputs = tf.contrib.layers.fully_connected(
    inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)  ## sequence_loss함수로 cost를 간단히 계산가능 
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.__call__` method instead.


In [8]:
#### running 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.674243 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
1 loss: 1.5711254 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
2 loss: 1.5051117 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
3 loss: 1.448404 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
4 loss: 1.3850368 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
5 loss: 1.3141826 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
6 loss: 1.2213479 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
7 loss: 1.1139129 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
8 loss: 1.0035089 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
9 loss: 0.8894078 prediction:  [[3 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predic